In [2]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [6]:
cd data


/Users/alexander/projects/DD2437/Lab2/data


In [11]:
animals = np.loadtxt(open("animals.dat", "rb"), delimiter=",", skiprows=0)

In [16]:
animals = np.reshape(animals,(32,84))

In [17]:
np.shape(animals)

(32, 84)